In [41]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
import requests
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from nltk.corpus import stopwords, reuters
import hvplot.pandas
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/samuelarciniega/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [42]:
load_dotenv()

True

In [43]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [44]:
api_key = os.getenv("news_api")
#print(api_key)

In [45]:
newsapi = NewsApiClient(api_key=api_key)

In [46]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
end_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["AAPL"]
apple_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=end_date,
    end=start_date,
    after=None,
    until=None,
).df
apple_ticker.dropna(inplace=True)
apple_ticker.head(50)

AAPL                                       
                             open     high       low     close    volume
time                                                                    
2021-06-07 00:00:00-04:00  126.18  126.320  124.8321  125.9100  62351961
2021-06-08 00:00:00-04:00  126.62  128.460  126.2101  126.7400  67231488
2021-06-09 00:00:00-04:00  127.21  127.750  126.5200  127.1600  46854275
2021-06-10 00:00:00-04:00  127.02  128.190  125.9400  126.1100  63377192
2021-06-11 00:00:00-04:00  126.53  127.440  126.1000  127.3521  46206013
2021-06-14 00:00:00-04:00  127.82  130.490  127.0700  130.4900  73169843
2021-06-15 00:00:00-04:00  129.88  130.600  129.3900  129.7400  53587032
2021-06-16 00:00:00-04:00  130.29  130.890  128.4610  130.1400  81577235
2021-06-17 00:00:00-04:00  129.80  132.550  129.6500  131.7800  82434812
2021-06-18 00:00:00-04:00  130.65  131.510  130.2400  130.4000  78114203
2021-06-21 00:00:00-04:00  130.32  132.410  129.2100  132.2900  63245869
2021-06-22 00:00:00-04:00  132.10  134.080  131.6200  133.9700  61664813
2021-06-23 00:00:00-04:00  133.78  134.320  133.2300  133.7000  54089974
2021-06-24 00:00:00-04:00  134.45  134.640  132.9300  133.4000  57263070
2021-06-25 00:00:00-04:00  133.47  133.890  132.8100  133.1100  42633100
2021-06-28 00:00:00-04:00  133.42  135.245  133.3500  134.7500  53815255
2021-06-29 00:00:00-04:00  134.82  136.490  134.3500  136.3300  55081959
2021-06-30 00:00:00-04:00  136.17  137.410  135.8700  136.9400  51014239
2021-07-01 00:00:00-04:00  136.60  137.330  135.7600  137.2700  46267213
2021-07-02 00:00:00-04:00  137.92  140.000  137.7450  139.9600  69160944

In [47]:
apple_df = pd.DataFrame()
apple_df["Apple_Close"] = apple_ticker["AAPL"]["close"]
#apple_df = apple_df.pct_change().dropna()
apple_df.index = apple_df.index.date 
apple_df.tail()

,Apple_Close
2021-06-28,134.75
2021-06-29,136.33
2021-06-30,136.94
2021-07-01,137.27
2021-07-02,139.96


In [48]:
apple_returns = apple_df.pct_change().dropna()
apple_returns.head(50)

,Apple_Close
2021-06-08,0.006592
2021-06-09,0.003314
2021-06-10,-0.008257
2021-06-11,0.009849
2021-06-14,0.024640
2021-06-15,-0.005748
2021-06-16,0.003083
2021-06-17,0.012602
2021-06-18,-0.010472
2021-06-21,0.014494


In [49]:
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(start_date[:10], "%Y-%m-%d")
    past_date = datetime.strptime(end_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > past_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
        
    return all_headlines, all_dates

In [50]:
#inflation_headlines, dates = get_headlines("inflation")

In [51]:
RegTech_headlines, dates = get_headlines("security")

Fetching news about 'security'
******************************
retrieving news from: 2021-07-05 00:00:00
retrieving news from: 2021-07-04 00:00:00
retrieving news from: 2021-07-03 00:00:00
retrieving news from: 2021-07-02 00:00:00
retrieving news from: 2021-07-01 00:00:00
retrieving news from: 2021-06-30 00:00:00
retrieving news from: 2021-06-29 00:00:00
retrieving news from: 2021-06-28 00:00:00
retrieving news from: 2021-06-27 00:00:00
retrieving news from: 2021-06-26 00:00:00
retrieving news from: 2021-06-25 00:00:00


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [13]:
#covid_headlines,_ = get_headlines("coronavirus")

In [14]:
sid = SentimentIntensityAnalyzer()

In [15]:
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [16]:
#inflation = headline_sentiment_summarizer_avg(inflation_headlines)
regtech = headline_sentiment_summarizer_avg(RegTech_headlines)
#regtech.to_csv("security.csv")
#covid = headline_sentiment_summarizer_avg(covid_headlines)

In [17]:
topic_sentiments = pd.DataFrame(
    {
        "Security_avg": regtech,
    }
)
#"Inflation_avg": inflation,
#"Security_avg": regtech,
#"Covid_avg": covid,
print(topic_sentiments)
#topic_sentiments.to_csv("test.csv")

    Security_avg
0      -0.129230
1      -0.024340
2      -0.185975
3      -0.068425
4       0.157910
5      -0.089700
6      -0.048560
7       0.030735
8      -0.031050
9      -0.073945
10     -0.029625
11     -0.154075
12      0.012780
13      0.091980
14      0.105125
15      0.009300
16      0.058350
17     -0.098280
18     -0.072455
19      0.018640
20      0.061830
21      0.173350
22     -0.007175
23     -0.111010
24      0.206090
25     -0.083360
26     -0.268535
27     -0.021220
28     -0.053740
29     -0.267230


In [18]:
import datetime
topic_sentiments.index = pd.to_datetime(dates)
topic_sentiments = apple_returns.join(topic_sentiments).dropna(how="any")
#topic_sentiments.to_csv("topic_sentiment.csv")
display(topic_sentiments)

,Apple_Close,Security_avg
2021-06-08,0.006592,-0.021220
2021-06-09,0.003314,-0.268535
2021-06-10,-0.008257,-0.083360
2021-06-11,0.009849,0.206090
2021-06-14,0.024640,0.173350
2021-06-15,-0.005748,0.061830
2021-06-16,0.003083,0.018640
2021-06-17,0.012602,-0.072455
2021-06-18,-0.010472,-0.098280
2021-06-21,0.014494,0.105125


In [19]:
topic_sentiments.corr().style.background_gradient()

,Apple_Close,Security_avg
Apple_Close,1.000000,0.379347
Security_avg,0.379347,1.000000


In [20]:
topic_sentiments.hvplot.line(figsize=(20,10))

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [37]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [38]:
window_size = 1
feature_column = 0
target_column = 1
X, y = window_data(topic_sentiments, window_size, feature_column, target_column)
#topic_sentiments.head()

In [39]:
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split :]
y_train = y[: split]
y_test = y[split :]

In [40]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [25]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print(f"X_train sample values:\n{X_train[:5]}\n")
print(f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.48599403]]

 [[0.3926305 ]]

 [[0.06307567]]

 [[0.57876483]]

 [[1.        ]]]

X_test sample values:
[[[0.24085008]]

 [[0.23434359]]

 [[0.23633486]]

 [[0.64914903]]

 [[0.63219679]]]


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [27]:
model = Sequential()
number_units = 5 
dropout_fraction = .2

#Layer1
model.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
#Layer2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
#Layer3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
#Output Layer
model.add(Dense(1))

In [28]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 5)              140       
_________________________________________________________________
dropout (Dropout)            (None, 1, 5)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 5)              220       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 5)              0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dropout_2 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 6

In [30]:
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
11/11 [==============================] - 3s 3ms/step - loss: 0.4241 
Epoch 2/10
11/11 [==============================] - 0s 2ms/step - loss: 0.4062
Epoch 3/10
11/11 [==============================] - 0s 3ms/step - loss: 0.3871
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 0.3715
Epoch 5/10
11/11 [==============================] - 0s 2ms/step - loss: 0.3469
Epoch 6/10
11/11 [==============================] - 0s 3ms/step - loss: 0.3287
Epoch 7/10
11/11 [==============================] - 0s 3ms/step - loss: 0.3096
Epoch 8/10
11/11 [==============================] - 0s 2ms/step - loss: 0.2870
Epoch 9/10
11/11 [==============================] - 0s 2ms/step - loss: 0.2759
Epoch 10/10
11/11 [==============================] - 0s 2ms/step - loss: 0.2580


In [31]:
model.evaluate(X_test, y_test)  

1/1 [==============================] - 1s 898ms/step - loss: 0.1564


0.15635451674461365

In [32]:
predicted = model.predict(X_test)

In [33]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [34]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = topic_sentiments.index[-len(real_prices): ]) 
stocks.head(100)

,Real,Predicted
2021-06-25,-0.154075,-0.182663
2021-06-28,-0.029625,-0.182685
2021-06-29,0.030735,-0.182678
2021-06-30,-0.048560,-0.181224
2021-07-01,-0.089700,-0.181285
2021-07-02,0.157910,-0.182022


In [36]:
stocks.hvplot.line(xlabel = "Date", ylabel= "Price")

:NdOverlay   [Variable]
   :Curve   [index]   (value)